In [ ]:
cd=r"C:\Users\HP\Desktop\jupyter project\url_folder\ url_folder\combined_urls_with_labels1.csv"

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
import re
from nltk.util import ngrams
from collections import Counter
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import image
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.metrics import ConfusionMatrixDisplay
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder




In [ ]:
# Function to extract character-based features
def extract_char_features(url):
    url = re.sub(r'^https?://', '', url)
    char_list = list(url)
    char_freq = Counter(char_list)
    return dict(char_freq)

# Function to extract n-gram features
def extract_ngrams_features(url, n=3):
    url = re.sub(r'^https?://', '', url)
    ngrams_list = list(ngrams(url, n))
    ngram_freq = Counter(ngrams_list)
    return dict(ngram_freq)

In [ ]:

# Path to the CSV file
cd=r"C:\Users\HP\Desktop\jupyter project\url_folder\ url_folder\combined_urls_with_labels1.csv"
df = pd.read_csv(cd)

# Assuming the CSV file has columns 'url' and 'label'
urls = df['url']
labels = df['label']

# Create lists to store the features and labels
char_features_list = []
ngram_features_list = []
labels_list = []

# Process each URL and extract features
for url, label in zip(urls, labels):
    # Extract character-based features
    char_features = extract_char_features(url)
    char_features_list.append(char_features)
    
    # Extract n-gram features (adjust n as necessary)
    ngram_features = extract_ngrams_features(url, n=3)
    ngram_features_list.append(ngram_features)
    
    # Store the label
    labels_list.append(label)

# Flatten character features and n-gram features into separate columns
char_df = pd.DataFrame(char_features_list)
ngram_df = pd.DataFrame(ngram_features_list)

# Ensure all DataFrames (character and n-gram features) have the same number of columns (use NaN for missing features)
char_df = char_df.fillna(0).astype(int)
ngram_df = ngram_df.fillna(0).astype(int)

# Combine the original URL and label columns with the feature DataFrames
feature_df = pd.concat([df[['url', 'label']], char_df, ngram_df], axis=1)

# Save to CSV
output_path = r"C:\Users\HP\Desktop\jupyter project\url_folder\url_folder\extracted_features.csv"
feature_df.to_csv(output_path, index=False)

print("Features saved successfully to", output_path)


In [ ]:


# Path to the CSV file where the extracted features are stored
input_csv = r"C:\Users\HP\Desktop\jupyter project\url_folder\url_folder\extracted_features.csv"

# Load the extracted features from the CSV file
df = pd.read_csv(input_csv)

# Check the first few rows of the DataFrame to ensure correct loading
print("Column Names:", df.columns)
print(df.head())

# Extract the features and labels
urls = df['url']
labels = df['label']

# For this case, features seem to be split into columns (like u, r, l, m, etc.)
# Assuming columns starting from index 2 are feature columns (adjust as needed)
feature_columns = df.columns[2:]

# Create a feature matrix from these columns
feature_matrix = df[feature_columns].values

# Directory to save the images
benign_folder = r"C:\Users\HP\Desktop\jupyter project\url_folder\url_img\benign_images"
non_benign_folder = r"C:\Users\HP\Desktop\jupyter project\url_folder\url_img\non_benign_images"

# Create directories if they do not exist
os.makedirs(benign_folder, exist_ok=True)
os.makedirs(non_benign_folder, exist_ok=True)

# Convert the features to vectors and reshape into images
for idx, (features, label) in enumerate(zip(feature_matrix, labels)):
    # Normalize the vector (min-max normalization to 0-255 range)
    normalized_features = 255 * (features - np.min(features)) / (np.max(features) - np.min(features))
    
    # Ensure the vector length is 4096
    if len(normalized_features) < 4096:
        # Pad with zeros if the vector is too short
        padded_features = np.pad(normalized_features, (0, 4096 - len(normalized_features)), mode='constant')
    else:
        # Truncate if the vector is too long
        padded_features = normalized_features[:4096]
    
    # Reshape the 4096-length vector into a 64x64 image
    reshaped_image = padded_features.reshape(64, 64)
    
    # Convert the 2D array into a grayscale image
    img = Image.fromarray(reshaped_image.astype(np.uint8))

    # Save the image to the appropriate folder based on the label
    if label == 0:  # Benign
        img.save(os.path.join(benign_folder, f"benign_image_{idx}.png"))
    else:  # Non-benign
        img.save(os.path.join(non_benign_folder, f"non_benign_image_{idx}.png"))

print("Grayscale images have been saved successfully.")


In [ ]:

benign_folder = r"C:\Users\HP\Desktop\jupyter project\url_folder\url_img\benign_images"
non_benign_folder = r"C:\Users\HP\Desktop\jupyter project\url_folder\url_img\non_benign_images"

In [ ]:

def load_images_and_labels(benign_folder, non_benign_folder):
    images = []
    labels = []
    
    # Load benign images
    for filename in os.listdir(benign_folder):
        if filename.endswith('.png'):
            img = np.array(Image.open(os.path.join(benign_folder, filename)))
            images.append(img)
            labels.append(0)  # Benign label
    
    # Load non-benign images
    for filename in os.listdir(non_benign_folder):
        if filename.endswith('.png'):
            img = np.array(Image.open(os.path.join(non_benign_folder, filename)))
            images.append(img)
            labels.append(1)  # Non-benign label

    return np.array(images), np.array(labels)

In [ ]:
X, y = load_images_and_labels(benign_folder, non_benign_folder)

# Preprocess data: Normalize and reshape for CNN input
X = X.astype('float32') / 255.0  # Normalize pixel values to [0, 1]
X = X.reshape(-1, 64, 64, 1)  # Reshape for CNN (64x64 pixels with 1 channel)

In [ ]:
# Encode labels to binary format (not one-hot encoding)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # This will convert labels to [0, 1]
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [ ]:
# Define the CNN model for feature extraction
cnn_model = Sequential()
cnn_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Flatten())
cnn_model.add(Dense(64, activation='relu'))

In [ ]:

# Compile the CNN model
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [ ]:

# Train the CNN model with validation data
history = cnn_model.fit(X_train, y_train, epochs=10, batch_size=32,
                        validation_data=(X_test, y_test))  # Include validation data


In [ ]:
# Extract features
cnn_model.trainable = False  # Freeze the CNN
train_features = cnn_model.predict(X_train)
test_features = cnn_model.predict(X_test)

In [ ]:

# ANN classifier
ann_model = Sequential([
    Dense(192, activation='relu', input_shape=(train_features.shape[1],)),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation="relu"),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])

In [ ]:
# Compile the ANN model
ann_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the ANN classifier on features extracted by CNN with validation data
history_ann = ann_model.fit(train_features, y_train, epochs=10, batch_size=32,
                            validation_data=(test_features, y_test))  # Include validation data

In [ ]:
# Evaluate the ANN classifier on test data
loss_ann, accuracy_ann = ann_model.evaluate(test_features, y_test)
print(f'Test Accuracy: {accuracy_ann * 100:.2f}%')

In [ ]:

# Print validation accuracy and loss from training history of ANN
print("Validation Loss:", history_ann.history['val_loss'])
print("Validation Accuracy:", history_ann.history['val_accuracy'])

In [ ]:


# Function to extract character-based features
def extract_char_features(url):
    url = re.sub(r'^https?://', '', url)  # Remove protocol
    char_list = list(url)
    char_freq = Counter(char_list)
    return dict(char_freq)

# Function to extract n-gram features
def extract_ngrams_features(url, n=3):
    url = re.sub(r'^https?://', '', url)  # Remove protocol
    return [''.join(gram) for gram in ngrams(url, n)]  # Return list of n-grams


In [ ]:
# Load the dataset
cd = r"C:\Users\HP\Desktop\jupyter project\url_folder\url_folder\combined_urls_with_labels1.csv"
df = pd.read_csv(cd)

# Assuming the CSV file has columns 'url' and 'label'
urls = df['url']
labels = df['label']

# Extract character-based features
char_features_list = []
for url in urls:
    char_features = extract_char_features(url)
    char_features_list.append(char_features)

# Convert character features to a DataFrame
char_df = pd.DataFrame(char_features_list).fillna(0).astype(int)

In [ ]:

# Extract n-gram strings for TF-IDF calculation
n = 3  # Set n for n-grams
ngram_strings = [' '.join(extract_ngrams_features(url, n)) for url in urls]

# Calculate TF-IDF for n-grams
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(ngram_strings)

# Convert TF-IDF matrix to DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Combine all features
final_feature_df = pd.concat([df[['url', 'label']], char_df, tfidf_df], axis=1)

# Save to CSV
output_path = r"C:\Users\HP\Desktop\jupyter project\url_folder\url_folder\extracted_features_with_tfidf.csv"
final_feature_df.to_csv(output_path, index=False)

print("Features with TF-IDF saved successfully to", output_path)

In [ ]:

# Load the CSV file
csv_path = r"C:\Users\HP\Desktop\jupyter project\url_folder\url_folder\extracted_features_with_tfidf.csv"
df = pd.read_csv(csv_path)

# Assuming the last column is the `label` and the rest are TF-IDF features
tfidf_matrix = df.iloc[:, :-1].values  # All columns except the last one
labels = df.iloc[:, -1].values  # The last column is the label


In [ ]:


# Assuming ngram_strings contains your n-gram features as strings
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(ngram_strings)  # This creates a sparse matrix
tfidf_array = tfidf_matrix.toarray()  # Convert sparse matrix to dense array
# If tfidf_matrix is already an array, just use it directly
tfidf_array = tfidf_matrix  # No conversion needed

In [ ]:


# Assuming `tfidf_matrix` (from previous steps) and `labels` are already defined
# Convert labels to integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(labels)  # 0 for benign, 1 for non-benign

# Pad TF-IDF matrix to make it square
tfidf_array = tfidf_matrix.toarray()
n_features = tfidf_array.shape[1]
square_size = int(np.ceil(np.sqrt(n_features)))  # Find the nearest square size
padded_array = np.pad(tfidf_array, ((0, 0), (0, square_size**2 - n_features)), 'constant')  # Pad with zeros
reshaped_array = padded_array.reshape(-1, square_size, square_size, 1)  # Reshape for CNN




In [ ]:

# Split into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(reshaped_array, y_encoded, test_size=0.2, random_state=42)

In [ ]:


# Build CNN model for feature extraction
cnn_model_txt = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(square_size, square_size, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
])

# Compile CNN
cnn_model_txt.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train CNN model
cnn_model_txt.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
# Extract features using the CNN
train_features = cnn_model_txt.predict(X_train)
test_features = cnn_model_txt.predict(X_test)

In [ ]:

# Build ANN Classifier
ann_model_txt = Sequential([
    Dense(192, activation='relu', input_dim=train_features.shape[1]),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation="relu"),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),  # Binary classification
])

# Compile ANN
ann_model_txt.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train ANN classifier
ann_model_txt.fit(train_features, y_train, epochs=10, batch_size=32, validation_data=(test_features, y_test))

In [ ]:



# Evaluate ANN on test data
loss, accuracy = ann_model_txt.evaluate(test_features, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [ ]:


# Remove the last classification layers from both CNN models
text_feature_extractor = Model(inputs=cnn_model_txt.input, outputs=cnn_model_txt.layers[-2].output)
image_feature_extractor = Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)





In [ ]:
# Assuming X_train_text and X_train_images are your input data for text and images
text_features_train = text_feature_extractor.predict(X_train)  # Features from text CNN
text_features_test = text_feature_extractor.predict(X_test)    # Test features for text CNN

In [ ]:

# Assuming X_train and X_test have shape (num_samples, 132, 132, 1)
input_shape = (64, 64)  # Target input shape for the CNN model

# Resize training and test images
X_train_resized = tf.image.resize(X_train, input_shape)
X_test_resized = tf.image.resize(X_test, input_shape)

# Ensure the shape matches the expected input shape of the model
X_train_resized = X_train_resized.numpy()  # Convert TensorFlow tensor to NumPy array if needed
X_test_resized = X_test_resized.numpy()

In [ ]:
image_features_train = image_feature_extractor.predict(X_train_resized)
image_features_test = image_feature_extractor.predict(X_test_resized)


In [ ]:

# Combine features from text and image models
train_features_combined = np.concatenate([text_features_train, image_features_train], axis=1)
test_features_combined = np.concatenate([text_features_test, image_features_test], axis=1)


In [ ]:
# Build ANN Classifier
hf_ann_model = Sequential([
    Dense(192, activation="relu", input_dim=train_features_combined.shape[1]),
    Dropout(0.5),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(64, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid"),  # Binary classification
])




In [ ]:
# Compile ANN Model
hf_ann_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train ANN Classifier
hf_ann_model.fit(train_features_combined, y_train, validation_data=(test_features_combined, y_test), epochs=10, batch_size=32)


In [ ]:

# Evaluate ANN
loss, accuracy = hf_ann_model.evaluate(test_features_combined, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [ ]:
# Predict on test data using combined features
y_pred_prob = hf_ann_model.predict(test_features_combined)  # Use combined features
y_pred = (y_pred_prob > 0.5).astype(int).flatten()  # Convert probabilities to binary predictions (0 or 1)

# Ensure true labels are correctly formatted
y_true = y_test.flatten()  # Flatten y_test if necessary (if it's one-dimensional already, this does nothing)

# Generate Confusion Matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Benign', 'Non-Benign'])
disp.plot(cmap='Blues')
plt.title('Confusion Matrix')
plt.show()
